In [1]:
# Copyright (c) 2020 Shubh Pachchigar
"""
EEG Data is taken from DEAP
The training data was taken from DEAP.
See my:
- Github profile: https://github.com/shubhe25p
- Email: shubhpachchigar@gmail.com
"""

import csv
import numpy as np
import scipy.spatial as ss
import scipy.stats as sst
import _pickle as cPickle
from pathlib import Path
import itertools
import random
import cv2
import scipy.io as sio


sampling_rate = 128
number_of_channel = 32  # considering only head electrodes
eeg_in_second = 63  # length of each trial
number_of_eeg = sampling_rate * eeg_in_second  # total inputs from a single channel

channel_names = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz',
                 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']


class predictEmotion(object):
    """
    Receives EEG data preprocessing and predict emotion.
    """

    # path is set to training data directory
    def __init__(self):
        """
        Initializes training data and their classes.
        """
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv")
        self.class_valence = self.get_csv("class_valence.csv")

    def get_csv(self, path):
        """
        Get data from csv and convert them to numpy python.
        Input: Path csv file.
        Output: Numpy array from csv data.
        """
        # Get csv data to list
        file_csv = open(path)
        data_csv = csv.reader(file_csv)
        # convert list to array with a specific dtype
        data_training = np.array(list(data_csv), dtype=np.float64)
        return data_training

    def do_fft(self, all_channel_data):
        """
        Do fft in each channel for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: FFT result with dimension N x M. N denotes number of channel and M denotes number of FFT data from each channel.
        """
        data_fft = map(lambda x: np.fft.fft(x), all_channel_data)

        return data_fft

    def get_frequency(self, all_channel_data):
        """
        Get frequency from computed fft for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Frequency band from each channel: Delta, Theta, Alpha, Beta, and Gamma.
        """
        # Length data channel
        L = len(all_channel_data[0])

        # Sampling frequency
        Fs = 128

        # Get fft data
        data_fft = self.do_fft(all_channel_data)

        # Compute frequency
        frequency = map(lambda x: abs(x // L), data_fft)
        frequency = map(lambda x: x[: L // 2 + 1] * 2, frequency)

        # creating 5 instances of frequency iterator
        f1, f2, f3, f4, f5 = itertools.tee(frequency, 5)

        # List frequency
        delta = np.array(list(map(lambda x: x[L * 1 // Fs - 1: L * 4 // Fs], f1)))
        theta = np.array(list(map(lambda x: x[L * 4 // Fs - 1: L * 8 // Fs], f2)))
        alpha = np.array(list(map(lambda x: x[L * 5 // Fs - 1: L * 13 // Fs], f3)))
        beta = np.array(list(map(lambda x: x[L * 13 // Fs - 1: L * 30 // Fs], f4)))
        gamma = np.array(list(map(lambda x: x[L * 30 // Fs - 1: L * 50 // Fs], f5)))

        return delta, theta, alpha, beta, gamma

    # def get_feature(self, all_channel_data):
    #     """
    #     Get feature from each frequency.
    #     Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
    #     Output: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
    #     """
    #
    #     (delta, theta, alpha, beta, gamma) = self.get_frequency(all_channel_data)
    #     delta_std = np.std(delta, axis=1)
    #     theta_std = np.std(theta, axis=1)
    #     alpha_std = np.std(alpha, axis=1)
    #     beta_std = np.std(beta, axis=1)
    #     gamma_std = np.std(gamma, axis=1)
    #     feature = np.array([delta_std, theta_std, alpha_std, beta_std, gamma_std])
    #     feature = feature.T
    #     feature = feature.ravel()
    #
    #     return feature
    def get_feature(self, all_channel_data): #수정
        """
        Get feature from each frequency.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Feature (standard deviation and mean) from all frequency bands and channels with dimension 1 x M (number of features).
        """
        # Extract frequency bands
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        # Compute standard deviation for each frequency band
        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        # Combine all features into a single array
        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature

    def get_frequency(self, all_channel_data):
        """
        Get frequency bands (Delta, Theta, Alpha, Beta, Gamma) from EEG data.
        Input: Channel data (N x M), where N is the number of channels and M is the number of samples.
        Output: Frequency bands for each channel: Delta, Theta, Alpha, Beta, Gamma.
        """
        # Number of samples (length of data in time)
        L = all_channel_data.shape[1]

        # Sampling frequency
        Fs = 128

        # Get FFT data
        data_fft = np.fft.fft(all_channel_data, axis=1)  # FFT along the sample axis

        # Compute frequency magnitudes
        frequency = np.abs(data_fft / L)  # Normalize by the length of the data
        frequency = frequency[:, : L // 2 + 1]  # Keep only positive frequencies

        # Frequency bins
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)  # Frequency values for bins

        # Extract frequency bands
        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma
    def predict_emotion(self, feature):
        """
        Get arousal and valence class from feature.
        Input: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 3 from each arousal and valence. 1 denotes low , 2 denotes neutral, and 3 denotes high .
        """
        # Compute canberra with arousal training data
        distance_ar = list(map(lambda x: ss.distance.canberra(x, feature), self.train_arousal))
        # Compute canberra with valence training data
        distance_va = list(map(lambda x: ss.distance.canberra(x, feature), self.train_valence))

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_ar = np.array(np.argsort(distance_ar)[:3])
        val_nearest_ar = np.array(np.sort(distance_ar)[:3])

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_va = np.array(np.argsort(distance_va)[:3])
        val_nearest_va = np.array(np.sort(distance_va)[:3])

        # Compute comparation from first nearest and second nearest distance. If comparation less or equal than 0.7, then take class from the first nearest distance. Else take frequently class.
        # Arousal
        comp_ar = val_nearest_ar[0] / val_nearest_ar[1]
        if comp_ar <= 0.7:
            result_ar = self.class_arousal[0, idx_nearest_ar[0]]
        else:
            result_ar = sst.mode(self.class_arousal[0, idx_nearest_ar])
            result_ar = float(result_ar[0])

        # Valence
        comp_va = val_nearest_va[0] / val_nearest_va[1]
        if comp_va <= 0.7:
            result_va = self.class_valence[0, idx_nearest_va[0]]
        else:
            result_va = sst.mode(self.class_valence[0, idx_nearest_va])
            result_va = float(result_va[0])

        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Get emotion class from feature.
        Input: Feature (standard deviasion) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Process all data from EEG data to predict emotion class.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model. And send it to web ap
        """
        # Get feature from EEG data
        feature = self.get_feature(all_channel_data)

        # Predict emotion class
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    #
    def send_result_to_window(self, emotion_class):
        """
        Send emotion predict to web app.
        Input: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        Output: Send emotion prediction to web window.
        """
        i1 = cv2.imread('emoji/1.png')
        i2 = cv2.imread('emoji/2.png')
        i3 = cv2.imread('emoji/3.png')
        i4 = cv2.imread('emoji/4.png')
        i5 = cv2.imread('emoji/5.png')
        if emotion_class == 1:
            cv2.imshow('image', i1)
        elif emotion_class == 2:
            cv2.imshow('image', i2)
        elif emotion_class == 3:
            cv2.imshow('image', i3)
        elif emotion_class == 4:
            cv2.imshow('image', i4)
        else:
            cv2.imshow('image', i5)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def main_process(self):
        """
        Input: Get EEG data from DEAP, process all data (FFT, feature extraction, and classification), and predict the emotion.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        # Load MATLAB .mat file
        fname = "../../DEAP/s03.mat"
        mat_data = sio.loadmat(fname)

        # Extract EEG data and labels
        eeg_data = mat_data['data']  # Update this key based on your .mat file structure
        labels = mat_data['labels']  # Update this key based on your .mat file structure

        # Check the structure of the loaded data
        print("Loaded EEG data shape:", eeg_data.shape)
        print("Loaded labels shape:", labels.shape)

        # Example: Use the 10th trial
        trial = 3 # Adjust trial index as needed
        eeg_realtime = eeg_data[trial, :, :]  # Assuming data is in [trials, channels, time] format
        print("Labels for trial:", labels[trial, :])  # Print labels for the selected trial

        # Use only the first 32 channels
        eeg_raw = eeg_realtime[:32, :]
        print("Raw EEG data shape (first 32 channels):", eeg_raw.shape)

        # Process and predict emotion
        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)
        print(mat_data.keys())
        print(mat_data['data'].shape)
        print(mat_data['labels'].shape)
        # Map emotion class to description
        if emotion_class == 1:
            print("Emotion: fear - nervous - stress - tense - upset")
        elif emotion_class == 2:
            print("Emotion: happy - alert - excited - elated")
        elif emotion_class == 3:
            print("Emotion: relax - calm - serene - contented")
        elif emotion_class == 4:
            print("Emotion: sad - depressed - lethargic - fatigue")
        else:
            print("Emotion: Neutral")

        # Display result in a window
        self.send_result_to_window(emotion_class)


#

if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()


Loaded EEG data shape: (40, 40, 8064)
Loaded labels shape: (40, 4)
Labels for trial: [4.69 3.69 5.   4.19]
Raw EEG data shape (first 32 channels): (32, 8064)
Extracted feature vector: [0.00925494 0.01727766 0.00793088 0.00901364 0.04197937 0.02121134
 0.05837321 0.02289057 0.04103291 0.0154061  0.04926387 0.03907766
 0.02298102 0.01019958 0.00741677 0.02310016 0.07752014 0.01177183
 0.02796507 0.02853318 0.03562128 0.04091504 0.02620511 0.01652603
 0.01449538 0.02958555 0.01507321 0.02489765 0.01538933 0.01206572
 0.04567938 0.01303984 0.06817449 0.07963289 0.05403238 0.06556241
 0.19411212 0.10812917 0.27150662 0.10568545 0.18755627 0.0752957
 0.21773402 0.1978261  0.13009162 0.0622614  0.05080129 0.10592977
 0.37332846 0.07838122 0.12076322 0.12940133 0.16915593 0.19040337
 0.11447951 0.0815081  0.07376492 0.13304952 0.08066141 0.1223634
 0.08596253 0.07735574 0.20594193 0.09012007 0.0408214  0.05314228
 0.04399428 0.03631615 0.11811252 0.06802233 0.17321525 0.06723102
 0.11819707 0.

2025-02-07 15:30:52.608 python[11158:9247843] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-07 15:30:52.608 python[11158:9247843] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [2]:

import csv
import numpy as np
import scipy.spatial as ss
import scipy.stats as sst
import _pickle as cPickle
from pathlib import Path
import itertools
import random
import cv2
import scipy.io as sio

In [3]:
from sklearn.neighbors import KNeighborsClassifier  # sklearn KNN 추가

In [4]:
sampling_rate = 128
number_of_channel = 32  # considering only head electrodes
eeg_in_second = 63  # length of each trial
number_of_eeg = sampling_rate * eeg_in_second  # total inputs from a single channel

channel_names = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz',
                 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']

In [5]:

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s03.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 3
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [4.69 3.69 5.   4.19]
Extracted feature vector: [[0.00925494 0.01727766 0.00793088 0.00901364 0.04197937 0.02121134
  0.05837321 0.02289057 0.04103291 0.0154061  0.04926387 0.03907766
  0.02298102 0.01019958 0.00741677 0.02310016 0.07752014 0.01177183
  0.02796507 0.02853318 0.03562128 0.04091504 0.02620511 0.01652603
  0.01449538 0.02958555 0.01507321 0.02489765 0.01538933 0.01206572
  0.04567938 0.01303984 0.06817449 0.07963289 0.05403238 0.06556241
  0.19411212 0.10812917 0.27150662 0.10568545 0.18755627 0.0752957
  0.21773402 0.1978261  0.13009162 0.0622614  0.05080129 0.10592977
  0.37332846 0.07838122 0.12076322 0.12940133 0.16915593 0.19040337
  0.11447951 0.0815081  0.07376492 0.13304952 0.08066141 0.1223634
  0.08596253 0.07735574 0.20594193 0.09012007 0.0408214  0.05314228
  0.04399428 0.03631615 0.11811252 0.06802233 0.17321525 0.06723102
  0.11819707 0.05077987 0.13340372 0.11647244 0.08645534 0.0480685
  0.03670386 0.06797771 0.22869211 0.04767986 0.08522

2025-02-07 15:34:13.346 python[11158:9247843] error messaging the mach port for IMKCFRunLoopWakeUpReliable


In [6]:

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s02.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 11
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [8.01 6.06 7.12 9.  ]
Extracted feature vector: [[0.17969023 0.41217685 0.03485786 0.1322709  0.08975296 0.02886256
  0.03819956 0.26620373 0.17533402 0.05434902 0.05308239 0.21896935
  0.02113338 0.2803271  0.14717365 0.01009093 0.07117237 0.12305581
  0.17056878 0.40954896 0.22322145 0.15183131 0.0808127  0.02722973
  0.05804292 0.08043799 0.16956451 0.08036793 0.13302882 0.00805446
  0.35794719 0.03753799 0.87735816 2.01102573 0.17566499 0.64979316
  0.44573774 0.17776601 0.18872944 1.31339155 0.86746764 0.26280657
  0.26204789 1.0628589  0.10489728 1.37595322 0.77038872 0.0491358
  0.35919841 0.60802085 0.84270998 2.00783848 1.10115639 0.74485814
  0.39209285 0.13633215 0.28803975 0.41073718 0.86685654 0.39851498
  0.65625567 0.0486425  1.78399797 0.18225606 0.23001062 0.53361318
  0.06189697 0.1773515  0.11681295 0.08679594 0.06222569 0.34750141
  0.22963592 0.08236435 0.07884199 0.28728734 0.06289973 0.36468279
  0.22226716 0.05051432 0.10679093 0.16494487 0.220

In [7]:

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s02.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 21
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [5.   1.   4.97 5.  ]
Extracted feature vector: [[0.24069318 0.55119687 0.04607098 0.17575102 0.12216078 0.03034242
  0.05115225 0.35628411 0.23369123 0.07262217 0.07038825 0.29669056
  0.02444525 0.37864719 0.19834433 0.01232335 0.09768915 0.16996091
  0.22794976 0.54844136 0.30534334 0.20486436 0.10530728 0.03743296
  0.07790033 0.10747362 0.23020086 0.10848818 0.17948618 0.00860996
  0.48275556 0.04956407 1.17259054 2.74947036 0.23337213 0.88463423
  0.60047874 0.15262896 0.25255316 1.78114355 1.17272091 0.36019354
  0.34883179 1.45029621 0.10857959 1.86968059 1.00276974 0.06717298
  0.49706824 0.83042558 1.1405035  2.72553274 1.50718558 1.01417861
  0.53159848 0.18009019 0.38708441 0.53327477 1.14553934 0.54033498
  0.89560426 0.04422877 2.40825222 0.23904652 0.34405196 0.78995047
  0.08921509 0.25570084 0.1764679  0.07841371 0.08114599 0.50931014
  0.34554461 0.10952369 0.11919339 0.4193721  0.06612792 0.54651991
  0.29632355 0.05159861 0.14573512 0.23532693 0.33

In [8]:

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s02.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 31
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [1.   9.   2.01 1.  ]
Extracted feature vector: [[0.18682523 0.43234821 0.03558512 0.13341085 0.09242939 0.02140605
  0.03826125 0.27483126 0.1798237  0.0576547  0.05305466 0.23507527
  0.02160102 0.29356232 0.13849339 0.00872551 0.07011404 0.13033147
  0.1775249  0.42884933 0.22677943 0.15858337 0.09004475 0.02704043
  0.05934116 0.08009544 0.16478182 0.08095695 0.13468802 0.00841596
  0.35974931 0.03918718 1.11073845 2.57979628 0.21223014 0.83075895
  0.56219366 0.1767153  0.23732645 1.66993683 1.10210358 0.33986944
  0.3287497  1.36366335 0.1154676  1.76680608 0.91767123 0.05858498
  0.44699206 0.76973561 1.07768002 2.56967859 1.39937181 0.94833142
  0.51154113 0.16524979 0.36230906 0.50102069 1.06231405 0.50458967
  0.83652155 0.05381337 2.24613908 0.23433003 0.30078781 0.69629989
  0.07065246 0.22074514 0.14698997 0.06704652 0.07206026 0.44318197
  0.29149015 0.10149982 0.09408473 0.38413245 0.0612792  0.46509893
  0.24383909 0.05285103 0.12385786 0.20883606 0.29

In [9]:

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s02.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 12
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [9.   4.99 9.   9.  ]
Extracted feature vector: [[0.18622754 0.4337156  0.03864159 0.14039307 0.09853161 0.02654487
  0.03890914 0.28167013 0.18642896 0.05840162 0.05583952 0.22701643
  0.01560532 0.29871634 0.16427488 0.0116993  0.07960688 0.13049993
  0.1839817  0.43267588 0.24561393 0.16085751 0.08190385 0.02819982
  0.0622688  0.09088115 0.18796906 0.08887138 0.14242899 0.00773472
  0.38750652 0.04352522 0.78296358 1.8294821  0.15629171 0.58891008
  0.40563864 0.14947946 0.16932781 1.18763505 0.77940913 0.23985443
  0.23516241 0.96610357 0.08733489 1.24612106 0.69734125 0.05009075
  0.33266146 0.55467558 0.75662775 1.81087397 1.00392856 0.68208352
  0.34783164 0.12778128 0.26490545 0.36735073 0.78326073 0.36072325
  0.6005537  0.05637681 1.62311451 0.16378946 0.20195519 0.43991993
  0.07659679 0.15829983 0.1055164  0.07882825 0.06582979 0.30198872
  0.20505247 0.08607924 0.09374708 0.23605006 0.07486929 0.31007228
  0.19684545 0.07170231 0.10113134 0.14005448 0.19

In [10]:

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s05.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 11
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [7.08 4.03 3.12 8.  ]
Extracted feature vector: [[0.06313443 0.04156552 0.04708774 0.09073143 0.02893664 0.05300306
  0.02246794 0.16360614 0.02238094 0.10213746 0.02012449 0.167994
  0.02176809 0.02622004 0.01759244 0.01769208 0.04725256 0.05713216
  0.11098698 0.03107602 0.04192809 0.09352368 0.01843674 0.1595773
  0.02359797 0.03701294 0.07102554 0.13179508 0.02457576 0.02543867
  0.05206858 0.09513737 0.26614954 0.19098641 0.19014148 0.39581043
  0.1312959  0.23640109 0.14376641 0.71126652 0.14268041 0.49311582
  0.12505384 0.75351166 0.13582672 0.12098869 0.07525446 0.11073205
  0.25953612 0.2639724  0.47977895 0.20683651 0.17018178 0.41795976
  0.08000504 0.84219764 0.12901406 0.21561213 0.31496918 0.61210351
  0.15994954 0.11450699 0.2207114  0.43012613 0.12514264 0.05741225
  0.09252628 0.14028972 0.07837474 0.12845514 0.07322206 0.24709028
  0.05869367 0.13149083 0.05236324 0.23960439 0.05471345 0.0414628
  0.03503372 0.05712588 0.14827528 0.07025861 0.162422

In [11]:
from sklearn.tree import DecisionTreeClassifier  # Decision Tree 추가

In [12]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s03.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 3
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [4.69 3.69 5.   4.19]
Extracted feature vector: [[0.00925494 0.01727766 0.00793088 0.00901364 0.04197937 0.02121134
  0.05837321 0.02289057 0.04103291 0.0154061  0.04926387 0.03907766
  0.02298102 0.01019958 0.00741677 0.02310016 0.07752014 0.01177183
  0.02796507 0.02853318 0.03562128 0.04091504 0.02620511 0.01652603
  0.01449538 0.02958555 0.01507321 0.02489765 0.01538933 0.01206572
  0.04567938 0.01303984 0.06817449 0.07963289 0.05403238 0.06556241
  0.19411212 0.10812917 0.27150662 0.10568545 0.18755627 0.0752957
  0.21773402 0.1978261  0.13009162 0.0622614  0.05080129 0.10592977
  0.37332846 0.07838122 0.12076322 0.12940133 0.16915593 0.19040337
  0.11447951 0.0815081  0.07376492 0.13304952 0.08066141 0.1223634
  0.08596253 0.07735574 0.20594193 0.09012007 0.0408214  0.05314228
  0.04399428 0.03631615 0.11811252 0.06802233 0.17321525 0.06723102
  0.11819707 0.05077987 0.13340372 0.11647244 0.08645534 0.0480685
  0.03670386 0.06797771 0.22869211 0.04767986 0.08522

In [13]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s03.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 33
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [4.72 8.12 5.85 5.33]
Extracted feature vector: [[0.02671391 0.04368588 0.01108576 0.01466029 0.14928385 0.03514972
  0.2130133  0.05458907 0.09973439 0.05329258 0.13509936 0.14878239
  0.08958897 0.02531842 0.00915001 0.07841941 0.27123928 0.01258355
  0.08265064 0.0766797  0.09813207 0.12172775 0.08193959 0.0524023
  0.03950008 0.10626139 0.05269783 0.09119869 0.05848134 0.03602908
  0.14717094 0.0242483  0.08293348 0.15341719 0.05918718 0.06793492
  0.46630071 0.15265491 0.64051689 0.189555   0.3578471  0.16329092
  0.45975708 0.45368753 0.27214667 0.07838481 0.04382452 0.23951572
  0.8601745  0.06914366 0.2537425  0.25227443 0.33814669 0.40050388
  0.24567806 0.14801628 0.13550162 0.31195692 0.16695937 0.2672594
  0.18244242 0.11957904 0.46086353 0.08166482 0.03899454 0.04840257
  0.03078653 0.0348978  0.12520128 0.06077923 0.17719222 0.06283165
  0.11792202 0.04682351 0.14053753 0.12568122 0.08227878 0.03717562
  0.03379686 0.06712143 0.24601893 0.03972496 0.0721

In [14]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s04.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 3
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [6.1  3.92 4.19 3.94]
Extracted feature vector: [[0.56397497 0.54301082 0.17662187 0.6440953  0.31901896 0.42386625
  0.73933286 0.08956844 0.20942445 0.35435936 0.05412845 0.10468765
  0.24391031 0.39968891 0.71627804 0.09914571 0.8935774  0.84435226
  0.38106309 0.69922621 0.15637192 0.57973274 0.09779149 0.1610675
  0.09351253 0.44069162 0.23062479 0.11189936 0.30158053 0.1485112
  0.2993232  0.12621696 2.17613271 2.07359093 0.68178393 2.4671866
  1.22206021 1.62561165 2.83937738 0.41164061 0.80178039 1.35867353
  0.21785282 0.41393358 0.9432397  1.54964746 2.74926887 0.37975675
  3.41993573 3.23521073 1.45608099 2.67016679 0.59209304 2.21906466
  0.37951053 0.62876082 0.35828234 1.69012318 0.8881968  0.44040749
  1.14361807 0.57469778 1.15019474 0.47396833 0.75906776 0.69976203
  0.23909841 0.85097407 0.40102292 0.54983745 0.93908884 0.16713352
  0.27601358 0.46677102 0.08044918 0.14697042 0.32680114 0.55647822
  0.92383767 0.14201876 1.18009248 1.08280843 0.49294

In [15]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s03.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 15
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [7.15 4.37 4.03 7.54]
Extracted feature vector: [[0.02048886 0.03133163 0.00940954 0.01576358 0.10872085 0.03309262
  0.15793872 0.04628223 0.08472071 0.04118071 0.10695959 0.10857484
  0.06724863 0.01835801 0.00762695 0.05465743 0.20271286 0.01241496
  0.06043216 0.06074042 0.07988723 0.0950192  0.06048897 0.0376208
  0.03182232 0.07635594 0.03984785 0.06974173 0.04511447 0.02788837
  0.10831748 0.01950472 0.10587106 0.17444298 0.06206217 0.07218389
  0.55740464 0.15192083 0.79029485 0.21764778 0.41057091 0.19646445
  0.54125287 0.55063623 0.3288287  0.08532916 0.04127382 0.29021245
  1.02103126 0.05161818 0.31147899 0.29971582 0.40167874 0.47689674
  0.30179522 0.1808346  0.15704969 0.38368552 0.20006921 0.32815614
  0.21724362 0.14054976 0.55660621 0.07488017 0.0389345  0.0563021
  0.03778722 0.03076534 0.13217945 0.05353008 0.18141029 0.06342585
  0.11890451 0.04729028 0.14569761 0.12592703 0.07241075 0.03586887
  0.03020172 0.07322037 0.24847253 0.03862114 0.0865

In [16]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s01.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 3
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [4.94 6.01 6.12 8.06]
Extracted feature vector: [[0.00721619 0.0071029  0.00765351 0.01002257 0.00428681 0.00596254
  0.00660091 0.00719939 0.00432849 0.00701936 0.01047084 0.0069911
  0.00793918 0.00721763 0.00697158 0.00632994 0.00939993 0.00690532
  0.00612207 0.00650098 0.01023378 0.00653775 0.00689821 0.00843214
  0.00464894 0.00781674 0.00496304 0.00440419 0.00871554 0.00490983
  0.00821044 0.00528762 0.04889184 0.06253852 0.06403549 0.06291715
  0.04805386 0.0440037  0.04442974 0.06744699 0.04405669 0.03044402
  0.06959624 0.05692249 0.04622638 0.04998336 0.05383314 0.03727329
  0.08474831 0.08452496 0.11024373 0.09715417 0.05271287 0.04899786
  0.09047936 0.06055466 0.04090397 0.07081647 0.0364722  0.02895308
  0.08101555 0.04200717 0.07142649 0.06045331 0.03997157 0.04095339
  0.04201341 0.04889208 0.03568986 0.03158028 0.0376398  0.04966377
  0.04480075 0.03172091 0.05308931 0.0549419  0.06551761 0.0553273
  0.05264431 0.04046016 0.04654245 0.03968809 0.0425

In [17]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s01.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 7
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [7.32 2.55 6.32 5.87]
Extracted feature vector: [[0.00697172 0.00826222 0.00886454 0.01021798 0.00843542 0.00544021
  0.00810881 0.01317469 0.00555651 0.0051197  0.00833278 0.00611414
  0.00467738 0.00665979 0.00565613 0.0058726  0.0095653  0.00818866
  0.00660132 0.00807275 0.00674776 0.00779366 0.00591185 0.00445907
  0.00822568 0.01035271 0.00464497 0.00487935 0.01208811 0.00498659
  0.00859785 0.00607233 0.05674297 0.06981202 0.06902561 0.06983105
  0.05296938 0.04557732 0.04708525 0.07462624 0.04346056 0.03170856
  0.07231519 0.05960713 0.04677641 0.05661923 0.06088474 0.03519927
  0.08500703 0.07389504 0.09628476 0.09043221 0.05442301 0.04397126
  0.07602886 0.03634407 0.04774181 0.06595115 0.03485828 0.02992195
  0.08412103 0.04525495 0.07624805 0.0666571  0.04909966 0.0548558
  0.05656424 0.0577868  0.04559487 0.04012018 0.04492599 0.05894761
  0.05704512 0.04035153 0.06488567 0.07130044 0.07850552 0.06148612
  0.06163066 0.05388786 0.05415009 0.04834795 0.058

In [18]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s11.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 11
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [5.04 1.01 7.46 1.37]
Extracted feature vector: [[0.10836293 0.14457994 0.16399681 0.71615736 0.18012011 0.10752067
  0.22933496 0.05325858 0.05407182 0.08600514 0.17107127 0.23764641
  0.11717368 0.32179423 0.28955243 0.06786123 0.15336309 0.03526876
  0.1732912  0.21578571 0.35077432 0.23004454 0.15363656 0.40793443
  0.50812901 0.31498017 0.45310678 0.20988443 0.04440102 0.23903681
  0.24440145 0.29193055 0.68787782 0.89758747 0.90264197 4.0644302
  0.98508006 0.59233027 1.33325852 0.30721714 0.33654557 0.4564336
  0.98903098 1.41812906 0.65465508 1.83282824 1.60432618 0.36719228
  0.95667325 0.17926976 0.94977573 1.23602005 2.05222005 1.35173408
  0.9132117  2.33173492 2.86617915 1.79343553 2.5649757  1.22901088
  0.24500051 1.35955342 1.39060487 1.65242832 0.24654666 0.35004108
  0.41903368 1.79827676 0.46068479 0.27272583 0.56950234 0.1103878
  0.1303751  0.20974802 0.41339256 0.57793168 0.28722718 0.79359043
  0.71664368 0.16439585 0.36540147 0.0841858  0.43341

In [19]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s14.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 5
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [7.19 6.83 6.55 7.67]
Extracted feature vector: [[0.02804191 0.01799224 0.21698544 0.02572117 0.01908772 0.01314259
  0.03159231 0.05327429 0.02384811 0.13196876 0.02331409 0.09654946
  0.02590371 0.03146604 0.01717981 0.02372317 0.20834162 0.09413624
  0.0289202  0.02436424 0.01821725 0.07439667 0.05916777 0.12182231
  0.05019999 0.10797308 0.0474547  0.12194556 0.1309736  0.06916956
  0.10781816 0.02541605 0.16286016 0.13701494 1.04299879 0.15528047
  0.11646407 0.08789889 0.16676766 0.30244562 0.14459665 0.62211502
  0.14696029 0.50548554 0.13631224 0.18923221 0.08921305 0.15764951
  0.91402383 0.42305998 0.19803667 0.16494764 0.11146414 0.39832124
  0.35475012 0.63612384 0.23372034 0.54065805 0.27294975 0.62081466
  0.59503534 0.35581075 0.48176463 0.14387292 0.10783911 0.08227654
  0.40219283 0.0832216  0.0749943  0.06353693 0.09068166 0.1368747
  0.08439841 0.25153635 0.10493167 0.20885976 0.09174673 0.10382152
  0.06191292 0.11759844 0.32789454 0.15672267 0.100

In [20]:
from sklearn.neighbors import KNeighborsClassifier  # sklearn KNN 추가

sampling_rate = 128
number_of_channel = 32  # considering only head electrodes
eeg_in_second = 63  # length of each trial
number_of_eeg = sampling_rate * eeg_in_second  # total inputs from a single channel

channel_names = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz',
                 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']

class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion.
    """

    def __init__(self):
        """
        Initializes training data and trains KNN classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize KNN classifiers
        self.knn_arousal = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
        self.knn_valence = KNeighborsClassifier(n_neighbors=3, metric="euclidean")

        # Train the models
        self.knn_arousal.fit(self.train_arousal, self.class_arousal)
        self.knn_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained KNN models.
        """
        result_ar = self.knn_arousal.predict(feature)[0]
        result_va = self.knn_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s02.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 12
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [9.   4.99 9.   9.  ]
Extracted feature vector: [[0.18622754 0.4337156  0.03864159 0.14039307 0.09853161 0.02654487
  0.03890914 0.28167013 0.18642896 0.05840162 0.05583952 0.22701643
  0.01560532 0.29871634 0.16427488 0.0116993  0.07960688 0.13049993
  0.1839817  0.43267588 0.24561393 0.16085751 0.08190385 0.02819982
  0.0622688  0.09088115 0.18796906 0.08887138 0.14242899 0.00773472
  0.38750652 0.04352522 0.78296358 1.8294821  0.15629171 0.58891008
  0.40563864 0.14947946 0.16932781 1.18763505 0.77940913 0.23985443
  0.23516241 0.96610357 0.08733489 1.24612106 0.69734125 0.05009075
  0.33266146 0.55467558 0.75662775 1.81087397 1.00392856 0.68208352
  0.34783164 0.12778128 0.26490545 0.36735073 0.78326073 0.36072325
  0.6005537  0.05637681 1.62311451 0.16378946 0.20195519 0.43991993
  0.07659679 0.15829983 0.1055164  0.07882825 0.06582979 0.30198872
  0.20505247 0.08607924 0.09374708 0.23605006 0.07486929 0.31007228
  0.19684545 0.07170231 0.10113134 0.14005448 0.19

In [21]:
class predictEmotion:
    """
    Receives EEG data preprocessing and predict emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data and classes
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # Convert to 1D array
        self.class_valence = self.get_csv("class_valence.csv").ravel()  # Convert to 1D array

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=5, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_arousal, self.class_arousal)
        self.tree_valence.fit(self.train_valence, self.class_valence)

    def get_csv(self, path):
        """
        Get data from CSV and convert it to a numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # Reshape for sklearn compatibility

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using the trained Decision Tree models.
        """
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def send_result_to_window(self, emotion_class):
        """
        Displays predicted emotion class in a window using OpenCV.
        """
        img_path = f'emoji/{emotion_class}.png'
        img = cv2.imread(img_path)
        if img is not None:
            cv2.imshow('Emotion Prediction', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print(f"Emotion class {emotion_class} image not found!")

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s02.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 12
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        self.send_result_to_window(emotion_class)


if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()

Labels for trial: [9.   4.99 9.   9.  ]
Extracted feature vector: [[0.18622754 0.4337156  0.03864159 0.14039307 0.09853161 0.02654487
  0.03890914 0.28167013 0.18642896 0.05840162 0.05583952 0.22701643
  0.01560532 0.29871634 0.16427488 0.0116993  0.07960688 0.13049993
  0.1839817  0.43267588 0.24561393 0.16085751 0.08190385 0.02819982
  0.0622688  0.09088115 0.18796906 0.08887138 0.14242899 0.00773472
  0.38750652 0.04352522 0.78296358 1.8294821  0.15629171 0.58891008
  0.40563864 0.14947946 0.16932781 1.18763505 0.77940913 0.23985443
  0.23516241 0.96610357 0.08733489 1.24612106 0.69734125 0.05009075
  0.33266146 0.55467558 0.75662775 1.81087397 1.00392856 0.68208352
  0.34783164 0.12778128 0.26490545 0.36735073 0.78326073 0.36072325
  0.6005537  0.05637681 1.62311451 0.16378946 0.20195519 0.43991993
  0.07659679 0.15829983 0.1055164  0.07882825 0.06582979 0.30198872
  0.20505247 0.08607924 0.09374708 0.23605006 0.07486929 0.31007228
  0.19684545 0.07170231 0.10113134 0.14005448 0.19

In [22]:

class predictEmotion(object):
    """
    Receives EEG data preprocessing and predict emotion.
    """

    # path is set to training data directory
    def __init__(self):
        """
        Initializes training data and their classes.
        """
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv")
        self.class_valence = self.get_csv("class_valence.csv")

    def get_csv(self, path):
        """
        Get data from csv and convert them to numpy python.
        Input: Path csv file.
        Output: Numpy array from csv data.
        """
        # Get csv data to list
        file_csv = open(path)
        data_csv = csv.reader(file_csv)
        # convert list to array with a specific dtype
        data_training = np.array(list(data_csv), dtype=np.float64)
        return data_training

    def do_fft(self, all_channel_data):
        """
        Do fft in each channel for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: FFT result with dimension N x M. N denotes number of channel and M denotes number of FFT data from each channel.
        """
        data_fft = map(lambda x: np.fft.fft(x), all_channel_data)

        return data_fft

    def get_frequency(self, all_channel_data):
        """
        Get frequency from computed fft for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Frequency band from each channel: Delta, Theta, Alpha, Beta, and Gamma.
        """
        # Length data channel
        L = len(all_channel_data[0])

        # Sampling frequency
        Fs = 128

        # Get fft data
        data_fft = self.do_fft(all_channel_data)

        # Compute frequency
        frequency = map(lambda x: abs(x // L), data_fft)
        frequency = map(lambda x: x[: L // 2 + 1] * 2, frequency)

        # creating 5 instances of frequency iterator
        f1, f2, f3, f4, f5 = itertools.tee(frequency, 5)

        # List frequency
        delta = np.array(list(map(lambda x: x[L * 1 // Fs - 1: L * 4 // Fs], f1)))
        theta = np.array(list(map(lambda x: x[L * 4 // Fs - 1: L * 8 // Fs], f2)))
        alpha = np.array(list(map(lambda x: x[L * 5 // Fs - 1: L * 13 // Fs], f3)))
        beta = np.array(list(map(lambda x: x[L * 13 // Fs - 1: L * 30 // Fs], f4)))
        gamma = np.array(list(map(lambda x: x[L * 30 // Fs - 1: L * 50 // Fs], f5)))

        return delta, theta, alpha, beta, gamma

    # def get_feature(self, all_channel_data):
    #     """
    #     Get feature from each frequency.
    #     Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
    #     Output: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
    #     """
    #
    #     (delta, theta, alpha, beta, gamma) = self.get_frequency(all_channel_data)
    #     delta_std = np.std(delta, axis=1)
    #     theta_std = np.std(theta, axis=1)
    #     alpha_std = np.std(alpha, axis=1)
    #     beta_std = np.std(beta, axis=1)
    #     gamma_std = np.std(gamma, axis=1)
    #     feature = np.array([delta_std, theta_std, alpha_std, beta_std, gamma_std])
    #     feature = feature.T
    #     feature = feature.ravel()
    #
    #     return feature
    def get_feature(self, all_channel_data): #수정
        """
        Get feature from each frequency.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Feature (standard deviation and mean) from all frequency bands and channels with dimension 1 x M (number of features).
        """
        # Extract frequency bands
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        # Compute standard deviation for each frequency band
        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        # Combine all features into a single array
        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature

    def get_frequency(self, all_channel_data):
        """
        Get frequency bands (Delta, Theta, Alpha, Beta, Gamma) from EEG data.
        Input: Channel data (N x M), where N is the number of channels and M is the number of samples.
        Output: Frequency bands for each channel: Delta, Theta, Alpha, Beta, Gamma.
        """
        # Number of samples (length of data in time)
        L = all_channel_data.shape[1]

        # Sampling frequency
        Fs = 128

        # Get FFT data
        data_fft = np.fft.fft(all_channel_data, axis=1)  # FFT along the sample axis

        # Compute frequency magnitudes
        frequency = np.abs(data_fft / L)  # Normalize by the length of the data
        frequency = frequency[:, : L // 2 + 1]  # Keep only positive frequencies

        # Frequency bins
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)  # Frequency values for bins

        # Extract frequency bands
        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma
    def predict_emotion(self, feature):
        """
        Get arousal and valence class from feature.
        Input: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 3 from each arousal and valence. 1 denotes low , 2 denotes neutral, and 3 denotes high .
        """
        # Compute canberra with arousal training data
        distance_ar = list(map(lambda x: ss.distance.canberra(x, feature), self.train_arousal))
        # Compute canberra with valence training data
        distance_va = list(map(lambda x: ss.distance.canberra(x, feature), self.train_valence))

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_ar = np.array(np.argsort(distance_ar)[:3])
        val_nearest_ar = np.array(np.sort(distance_ar)[:3])

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_va = np.array(np.argsort(distance_va)[:3])
        val_nearest_va = np.array(np.sort(distance_va)[:3])

        # Compute comparation from first nearest and second nearest distance. If comparation less or equal than 0.7, then take class from the first nearest distance. Else take frequently class.
        # Arousal
        comp_ar = val_nearest_ar[0] / val_nearest_ar[1]
        if comp_ar <= 0.7:
            result_ar = self.class_arousal[0, idx_nearest_ar[0]]
        else:
            result_ar = sst.mode(self.class_arousal[0, idx_nearest_ar])
            result_ar = float(result_ar[0])

        # Valence
        comp_va = val_nearest_va[0] / val_nearest_va[1]
        if comp_va <= 0.7:
            result_va = self.class_valence[0, idx_nearest_va[0]]
        else:
            result_va = sst.mode(self.class_valence[0, idx_nearest_va])
            result_va = float(result_va[0])

        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Get emotion class from feature.
        Input: Feature (standard deviasion) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Process all data from EEG data to predict emotion class.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model. And send it to web ap
        """
        # Get feature from EEG data
        feature = self.get_feature(all_channel_data)

        # Predict emotion class
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    #
    def send_result_to_window(self, emotion_class):
        """
        Send emotion predict to web app.
        Input: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        Output: Send emotion prediction to web window.
        """
        i1 = cv2.imread('emoji/1.png')
        i2 = cv2.imread('emoji/2.png')
        i3 = cv2.imread('emoji/3.png')
        i4 = cv2.imread('emoji/4.png')
        i5 = cv2.imread('emoji/5.png')
        if emotion_class == 1:
            cv2.imshow('image', i1)
        elif emotion_class == 2:
            cv2.imshow('image', i2)
        elif emotion_class == 3:
            cv2.imshow('image', i3)
        elif emotion_class == 4:
            cv2.imshow('image', i4)
        else:
            cv2.imshow('image', i5)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def main_process(self):
        """
        Input: Get EEG data from DEAP, process all data (FFT, feature extraction, and classification), and predict the emotion.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        # Load MATLAB .mat file
        fname = "../../DEAP/s01.mat"
        mat_data = sio.loadmat(fname)

        # Extract EEG data and labels
        eeg_data = mat_data['data']  # Update this key based on your .mat file structure
        labels = mat_data['labels']  # Update this key based on your .mat file structure

        # Check the structure of the loaded data
        print("Loaded EEG data shape:", eeg_data.shape)
        print("Loaded labels shape:", labels.shape)

        # Example: Use the 10th trial
        trial = 1 # Adjust trial index as needed
        eeg_realtime = eeg_data[trial, :, :]  # Assuming data is in [trials, channels, time] format
        print("Labels for trial:", labels[trial, :])  # Print labels for the selected trial

        # Use only the first 32 channels
        eeg_raw = eeg_realtime[:32, :]
        print("Raw EEG data shape (first 32 channels):", eeg_raw.shape)

        # Process and predict emotion
        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)
        print(mat_data.keys())
        print(mat_data['data'].shape)
        print(mat_data['labels'].shape)
        # Map emotion class to description
        if emotion_class == 1:
            print("Emotion: fear - nervous - stress - tense - upset")
        elif emotion_class == 2:
            print("Emotion: happy - alert - excited - elated")
        elif emotion_class == 3:
            print("Emotion: relax - calm - serene - contented")
        elif emotion_class == 4:
            print("Emotion: sad - depressed - lethargic - fatigue")
        else:
            print("Emotion: Neutral")

        # Display result in a window
        self.send_result_to_window(emotion_class)


#

if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()


Loaded EEG data shape: (40, 40, 8064)
Loaded labels shape: (40, 4)
Labels for trial: [8.1  7.31 7.28 8.47]
Raw EEG data shape (first 32 channels): (32, 8064)
Extracted feature vector: [0.00826119 0.00894245 0.00865477 0.00968921 0.00949899 0.0078266
 0.00613189 0.01297476 0.00665116 0.00519848 0.01175978 0.00642798
 0.00882892 0.00856287 0.00875766 0.00667839 0.01136374 0.01239979
 0.00813205 0.01108761 0.00885692 0.0062595  0.00755456 0.00402511
 0.00675133 0.0093219  0.00463137 0.00543166 0.01309381 0.00610728
 0.01103936 0.00840796 0.05003394 0.05915513 0.06038345 0.06288535
 0.0469807  0.04558712 0.04585895 0.06841661 0.04028793 0.03424536
 0.07062238 0.05645964 0.04625136 0.05653538 0.06019559 0.03895279
 0.07389829 0.06195827 0.07098107 0.07373103 0.05685939 0.04401173
 0.06295797 0.03552709 0.04194018 0.06213391 0.03182029 0.03221473
 0.07665831 0.04248446 0.06983171 0.06375327 0.04166241 0.04190214
 0.04394802 0.05201548 0.03841169 0.03290772 0.042694   0.05515778
 0.04655522 0

In [24]:

class predictEmotion(object):
    """
    Receives EEG data preprocessing and predict emotion.
    """

    # path is set to training data directory
    def __init__(self):
        """
        Initializes training data and their classes.
        """
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv")
        self.class_valence = self.get_csv("class_valence.csv")

    def get_csv(self, path):
        """
        Get data from csv and convert them to numpy python.
        Input: Path csv file.
        Output: Numpy array from csv data.
        """
        # Get csv data to list
        file_csv = open(path)
        data_csv = csv.reader(file_csv)
        # convert list to array with a specific dtype
        data_training = np.array(list(data_csv), dtype=np.float64)
        return data_training

    def do_fft(self, all_channel_data):
        """
        Do fft in each channel for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: FFT result with dimension N x M. N denotes number of channel and M denotes number of FFT data from each channel.
        """
        data_fft = map(lambda x: np.fft.fft(x), all_channel_data)

        return data_fft

    def get_frequency(self, all_channel_data):
        """
        Get frequency from computed fft for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Frequency band from each channel: Delta, Theta, Alpha, Beta, and Gamma.
        """
        # Length data channel
        L = len(all_channel_data[0])

        # Sampling frequency
        Fs = 128

        # Get fft data
        data_fft = self.do_fft(all_channel_data)

        # Compute frequency
        frequency = map(lambda x: abs(x // L), data_fft)
        frequency = map(lambda x: x[: L // 2 + 1] * 2, frequency)

        # creating 5 instances of frequency iterator
        f1, f2, f3, f4, f5 = itertools.tee(frequency, 5)

        # List frequency
        delta = np.array(list(map(lambda x: x[L * 1 // Fs - 1: L * 4 // Fs], f1)))
        theta = np.array(list(map(lambda x: x[L * 4 // Fs - 1: L * 8 // Fs], f2)))
        alpha = np.array(list(map(lambda x: x[L * 5 // Fs - 1: L * 13 // Fs], f3)))
        beta = np.array(list(map(lambda x: x[L * 13 // Fs - 1: L * 30 // Fs], f4)))
        gamma = np.array(list(map(lambda x: x[L * 30 // Fs - 1: L * 50 // Fs], f5)))

        return delta, theta, alpha, beta, gamma

    # def get_feature(self, all_channel_data):
    #     """
    #     Get feature from each frequency.
    #     Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
    #     Output: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
    #     """
    #
    #     (delta, theta, alpha, beta, gamma) = self.get_frequency(all_channel_data)
    #     delta_std = np.std(delta, axis=1)
    #     theta_std = np.std(theta, axis=1)
    #     alpha_std = np.std(alpha, axis=1)
    #     beta_std = np.std(beta, axis=1)
    #     gamma_std = np.std(gamma, axis=1)
    #     feature = np.array([delta_std, theta_std, alpha_std, beta_std, gamma_std])
    #     feature = feature.T
    #     feature = feature.ravel()
    #
    #     return feature
    def get_feature(self, all_channel_data): #수정
        """
        Get feature from each frequency.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Feature (standard deviation and mean) from all frequency bands and channels with dimension 1 x M (number of features).
        """
        # Extract frequency bands
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        # Compute standard deviation for each frequency band
        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        # Combine all features into a single array
        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature

    def get_frequency(self, all_channel_data):
        """
        Get frequency bands (Delta, Theta, Alpha, Beta, Gamma) from EEG data.
        Input: Channel data (N x M), where N is the number of channels and M is the number of samples.
        Output: Frequency bands for each channel: Delta, Theta, Alpha, Beta, Gamma.
        """
        # Number of samples (length of data in time)
        L = all_channel_data.shape[1]

        # Sampling frequency
        Fs = 128

        # Get FFT data
        data_fft = np.fft.fft(all_channel_data, axis=1)  # FFT along the sample axis

        # Compute frequency magnitudes
        frequency = np.abs(data_fft / L)  # Normalize by the length of the data
        frequency = frequency[:, : L // 2 + 1]  # Keep only positive frequencies

        # Frequency bins
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)  # Frequency values for bins

        # Extract frequency bands
        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma
    def predict_emotion(self, feature):
        """
        Get arousal and valence class from feature.
        Input: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 3 from each arousal and valence. 1 denotes low , 2 denotes neutral, and 3 denotes high .
        """
        # Compute canberra with arousal training data
        distance_ar = list(map(lambda x: ss.distance.canberra(x, feature), self.train_arousal))
        # Compute canberra with valence training data
        distance_va = list(map(lambda x: ss.distance.canberra(x, feature), self.train_valence))

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_ar = np.array(np.argsort(distance_ar)[:3])
        val_nearest_ar = np.array(np.sort(distance_ar)[:3])

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_va = np.array(np.argsort(distance_va)[:3])
        val_nearest_va = np.array(np.sort(distance_va)[:3])

        # Compute comparation from first nearest and second nearest distance. If comparation less or equal than 0.7, then take class from the first nearest distance. Else take frequently class.
        # Arousal
        comp_ar = val_nearest_ar[0] / val_nearest_ar[1]
        if comp_ar <= 0.7:
            result_ar = self.class_arousal[0, idx_nearest_ar[0]]
        else:
            result_ar = sst.mode(self.class_arousal[0, idx_nearest_ar])
            result_ar = float(result_ar[0])

        # Valence
        comp_va = val_nearest_va[0] / val_nearest_va[1]
        if comp_va <= 0.7:
            result_va = self.class_valence[0, idx_nearest_va[0]]
        else:
            result_va = sst.mode(self.class_valence[0, idx_nearest_va])
            result_va = float(result_va[0])

        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Get emotion class from feature.
        Input: Feature (standard deviasion) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Process all data from EEG data to predict emotion class.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model. And send it to web ap
        """
        # Get feature from EEG data
        feature = self.get_feature(all_channel_data)

        # Predict emotion class
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    #
    def send_result_to_window(self, emotion_class):
        """
        Send emotion predict to web app.
        Input: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        Output: Send emotion prediction to web window.
        """
        i1 = cv2.imread('emoji/1.png')
        i2 = cv2.imread('emoji/2.png')
        i3 = cv2.imread('emoji/3.png')
        i4 = cv2.imread('emoji/4.png')
        i5 = cv2.imread('emoji/5.png')
        if emotion_class == 1:
            cv2.imshow('image', i1)
        elif emotion_class == 2:
            cv2.imshow('image', i2)
        elif emotion_class == 3:
            cv2.imshow('image', i3)
        elif emotion_class == 4:
            cv2.imshow('image', i4)
        else:
            cv2.imshow('image', i5)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def main_process(self):
        """
        Input: Get EEG data from DEAP, process all data (FFT, feature extraction, and classification), and predict the emotion.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        # Load MATLAB .mat file
        fname = "../../DEAP/s01.mat"
        mat_data = sio.loadmat(fname)

        # Extract EEG data and labels
        eeg_data = mat_data['data']  # Update this key based on your .mat file structure
        labels = mat_data['labels']  # Update this key based on your .mat file structure

        # Check the structure of the loaded data
        print("Loaded EEG data shape:", eeg_data.shape)
        print("Loaded labels shape:", labels.shape)

        # Example: Use the 10th trial
        trial = 34 # Adjust trial index as needed
        eeg_realtime = eeg_data[trial, :, :]  # Assuming data is in [trials, channels, time] format
        print("Labels for trial:", labels[trial, :])  # Print labels for the selected trial

        # Use only the first 32 channels
        eeg_raw = eeg_realtime[:32, :]
        print("Raw EEG data shape (first 32 channels):", eeg_raw.shape)

        # Process and predict emotion
        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)
        print(mat_data.keys())
        print(mat_data['data'].shape)
        print(mat_data['labels'].shape)
        # Map emotion class to description
        if emotion_class == 1:
            print("Emotion: fear - nervous - stress - tense - upset")
        elif emotion_class == 2:
            print("Emotion: happy - alert - excited - elated")
        elif emotion_class == 3:
            print("Emotion: relax - calm - serene - contented")
        elif emotion_class == 4:
            print("Emotion: sad - depressed - lethargic - fatigue")
        else:
            print("Emotion: Neutral")

        # Display result in a window
        self.send_result_to_window(emotion_class)


#

if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()


Loaded EEG data shape: (40, 40, 8064)
Loaded labels shape: (40, 4)
Labels for trial: [2.06 8.15 8.05 5.18]
Raw EEG data shape (first 32 channels): (32, 8064)
Extracted feature vector: [0.00903067 0.01387472 0.0145807  0.01118732 0.01196769 0.0104096
 0.0091783  0.01537518 0.00704191 0.0046463  0.01756914 0.00705344
 0.00584984 0.00495342 0.00579871 0.00393305 0.01949907 0.0091541
 0.01023626 0.01527191 0.01121554 0.00639593 0.01301983 0.00592374
 0.01092335 0.01424364 0.00482274 0.00452325 0.02269827 0.00423074
 0.0164139  0.00899412 0.04138752 0.0496746  0.05829236 0.05612059
 0.04359718 0.03829629 0.0476641  0.07035298 0.0367814  0.02663225
 0.06579617 0.04187141 0.03990143 0.04197876 0.04538896 0.02867218
 0.06190886 0.05326928 0.0550414  0.06113934 0.04567418 0.03708106
 0.05669906 0.03352843 0.04019245 0.06096789 0.02680832 0.02857003
 0.07916803 0.03219565 0.06289571 0.05025836 0.03764298 0.03831889
 0.03873292 0.04349123 0.03349002 0.03068139 0.03631918 0.04522284
 0.04426036 0.

In [25]:

class predictEmotion(object):
    """
    Receives EEG data preprocessing and predict emotion.
    """

    # path is set to training data directory
    def __init__(self):
        """
        Initializes training data and their classes.
        """
        self.train_arousal = self.get_csv("train_std.csv")
        self.train_valence = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv")
        self.class_valence = self.get_csv("class_valence.csv")

    def get_csv(self, path):
        """
        Get data from csv and convert them to numpy python.
        Input: Path csv file.
        Output: Numpy array from csv data.
        """
        # Get csv data to list
        file_csv = open(path)
        data_csv = csv.reader(file_csv)
        # convert list to array with a specific dtype
        data_training = np.array(list(data_csv), dtype=np.float64)
        return data_training

    def do_fft(self, all_channel_data):
        """
        Do fft in each channel for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: FFT result with dimension N x M. N denotes number of channel and M denotes number of FFT data from each channel.
        """
        data_fft = map(lambda x: np.fft.fft(x), all_channel_data)

        return data_fft

    def get_frequency(self, all_channel_data):
        """
        Get frequency from computed fft for all channels.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Frequency band from each channel: Delta, Theta, Alpha, Beta, and Gamma.
        """
        # Length data channel
        L = len(all_channel_data[0])

        # Sampling frequency
        Fs = 128

        # Get fft data
        data_fft = self.do_fft(all_channel_data)

        # Compute frequency
        frequency = map(lambda x: abs(x // L), data_fft)
        frequency = map(lambda x: x[: L // 2 + 1] * 2, frequency)

        # creating 5 instances of frequency iterator
        f1, f2, f3, f4, f5 = itertools.tee(frequency, 5)

        # List frequency
        delta = np.array(list(map(lambda x: x[L * 1 // Fs - 1: L * 4 // Fs], f1)))
        theta = np.array(list(map(lambda x: x[L * 4 // Fs - 1: L * 8 // Fs], f2)))
        alpha = np.array(list(map(lambda x: x[L * 5 // Fs - 1: L * 13 // Fs], f3)))
        beta = np.array(list(map(lambda x: x[L * 13 // Fs - 1: L * 30 // Fs], f4)))
        gamma = np.array(list(map(lambda x: x[L * 30 // Fs - 1: L * 50 // Fs], f5)))

        return delta, theta, alpha, beta, gamma

    # def get_feature(self, all_channel_data):
    #     """
    #     Get feature from each frequency.
    #     Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
    #     Output: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
    #     """
    #
    #     (delta, theta, alpha, beta, gamma) = self.get_frequency(all_channel_data)
    #     delta_std = np.std(delta, axis=1)
    #     theta_std = np.std(theta, axis=1)
    #     alpha_std = np.std(alpha, axis=1)
    #     beta_std = np.std(beta, axis=1)
    #     gamma_std = np.std(gamma, axis=1)
    #     feature = np.array([delta_std, theta_std, alpha_std, beta_std, gamma_std])
    #     feature = feature.T
    #     feature = feature.ravel()
    #
    #     return feature
    def get_feature(self, all_channel_data): #수정
        """
        Get feature from each frequency.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Feature (standard deviation and mean) from all frequency bands and channels with dimension 1 x M (number of features).
        """
        # Extract frequency bands
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        # Compute standard deviation for each frequency band
        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        # Combine all features into a single array
        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature

    def get_frequency(self, all_channel_data):
        """
        Get frequency bands (Delta, Theta, Alpha, Beta, Gamma) from EEG data.
        Input: Channel data (N x M), where N is the number of channels and M is the number of samples.
        Output: Frequency bands for each channel: Delta, Theta, Alpha, Beta, Gamma.
        """
        # Number of samples (length of data in time)
        L = all_channel_data.shape[1]

        # Sampling frequency
        Fs = 128

        # Get FFT data
        data_fft = np.fft.fft(all_channel_data, axis=1)  # FFT along the sample axis

        # Compute frequency magnitudes
        frequency = np.abs(data_fft / L)  # Normalize by the length of the data
        frequency = frequency[:, : L // 2 + 1]  # Keep only positive frequencies

        # Frequency bins
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)  # Frequency values for bins

        # Extract frequency bands
        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma
    def predict_emotion(self, feature):
        """
        Get arousal and valence class from feature.
        Input: Feature (standard deviasion and mean) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 3 from each arousal and valence. 1 denotes low , 2 denotes neutral, and 3 denotes high .
        """
        # Compute canberra with arousal training data
        distance_ar = list(map(lambda x: ss.distance.canberra(x, feature), self.train_arousal))
        # Compute canberra with valence training data
        distance_va = list(map(lambda x: ss.distance.canberra(x, feature), self.train_valence))

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_ar = np.array(np.argsort(distance_ar)[:3])
        val_nearest_ar = np.array(np.sort(distance_ar)[:3])

        # Compute 3 nearest index and distance value from arousal
        idx_nearest_va = np.array(np.argsort(distance_va)[:3])
        val_nearest_va = np.array(np.sort(distance_va)[:3])

        # Compute comparation from first nearest and second nearest distance. If comparation less or equal than 0.7, then take class from the first nearest distance. Else take frequently class.
        # Arousal
        comp_ar = val_nearest_ar[0] / val_nearest_ar[1]
        if comp_ar <= 0.7:
            result_ar = self.class_arousal[0, idx_nearest_ar[0]]
        else:
            result_ar = sst.mode(self.class_arousal[0, idx_nearest_ar])
            result_ar = float(result_ar[0])

        # Valence
        comp_va = val_nearest_va[0] / val_nearest_va[1]
        if comp_va <= 0.7:
            result_va = self.class_valence[0, idx_nearest_va[0]]
        else:
            result_va = sst.mode(self.class_valence[0, idx_nearest_va])
            result_va = float(result_va[0])

        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Get emotion class from feature.
        Input: Feature (standard deviasion) from all frequency bands and channels with dimesion 1 x M (number of feature).
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Process all data from EEG data to predict emotion class.
        Input: Channel data with dimension N x M. N denotes number of channel and M denotes number of EEG data from each channel.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model. And send it to web ap
        """
        # Get feature from EEG data
        feature = self.get_feature(all_channel_data)

        # Predict emotion class
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    #
    def send_result_to_window(self, emotion_class):
        """
        Send emotion predict to web app.
        Input: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        Output: Send emotion prediction to web window.
        """
        i1 = cv2.imread('emoji/1.png')
        i2 = cv2.imread('emoji/2.png')
        i3 = cv2.imread('emoji/3.png')
        i4 = cv2.imread('emoji/4.png')
        i5 = cv2.imread('emoji/5.png')
        if emotion_class == 1:
            cv2.imshow('image', i1)
        elif emotion_class == 2:
            cv2.imshow('image', i2)
        elif emotion_class == 3:
            cv2.imshow('image', i3)
        elif emotion_class == 4:
            cv2.imshow('image', i4)
        else:
            cv2.imshow('image', i5)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def main_process(self):
        """
        Input: Get EEG data from DEAP, process all data (FFT, feature extraction, and classification), and predict the emotion.
        Output: Class of emotion between 1 to 5 according to Russel's Circumplex Model.
        """
        # Load MATLAB .mat file
        fname = "../../DEAP/s01.mat"
        mat_data = sio.loadmat(fname)

        # Extract EEG data and labels
        eeg_data = mat_data['data']  # Update this key based on your .mat file structure
        labels = mat_data['labels']  # Update this key based on your .mat file structure

        # Check the structure of the loaded data
        print("Loaded EEG data shape:", eeg_data.shape)
        print("Loaded labels shape:", labels.shape)

        # Example: Use the 10th trial
        trial = 5 # Adjust trial index as needed
        eeg_realtime = eeg_data[trial, :, :]  # Assuming data is in [trials, channels, time] format
        print("Labels for trial:", labels[trial, :])  # Print labels for the selected trial

        # Use only the first 32 channels
        eeg_raw = eeg_realtime[:32, :]
        print("Raw EEG data shape (first 32 channels):", eeg_raw.shape)

        # Process and predict emotion
        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)
        print(mat_data.keys())
        print(mat_data['data'].shape)
        print(mat_data['labels'].shape)
        # Map emotion class to description
        if emotion_class == 1:
            print("Emotion: fear - nervous - stress - tense - upset")
        elif emotion_class == 2:
            print("Emotion: happy - alert - excited - elated")
        elif emotion_class == 3:
            print("Emotion: relax - calm - serene - contented")
        elif emotion_class == 4:
            print("Emotion: sad - depressed - lethargic - fatigue")
        else:
            print("Emotion: Neutral")

        # Display result in a window
        self.send_result_to_window(emotion_class)


#

if __name__ == "__main__":
    rte = predictEmotion()
    rte.main_process()


Loaded EEG data shape: (40, 40, 8064)
Loaded labels shape: (40, 4)
Labels for trial: [8.27 3.92 7.   8.03]
Raw EEG data shape (first 32 channels): (32, 8064)
Extracted feature vector: [0.00552933 0.0069489  0.00656479 0.00618776 0.00760975 0.00705758
 0.00717399 0.0108008  0.00706365 0.00338879 0.00955707 0.00708776
 0.00677929 0.00704739 0.00604832 0.00483437 0.01058658 0.00812481
 0.00553268 0.00997987 0.0085093  0.00643946 0.0058053  0.00397783
 0.00603265 0.00809184 0.00466438 0.00334004 0.01111384 0.00373613
 0.00882658 0.006033   0.04364944 0.04676025 0.05028588 0.05452113
 0.03974645 0.03956341 0.04080171 0.05582009 0.03612162 0.03051118
 0.05450318 0.04335168 0.04692972 0.04932737 0.04776607 0.0317056
 0.05751495 0.05336224 0.05259814 0.05561269 0.04709865 0.0383923
 0.05217077 0.03785619 0.039265   0.05288162 0.0285827  0.03072955
 0.063055   0.03736115 0.05966765 0.05715222 0.04224993 0.03978972
 0.03933859 0.04664077 0.0402185  0.02867943 0.04000208 0.05120096
 0.04945283 0.

In [26]:
# Copyright (c) 2020 Shubh Pachchigar
"""
EEG Data is taken from DEAP
The training data was taken from DEAP.
See my:
- Github profile: https://github.com/shubhe25p
- Email: shubhpachchigar@gmail.com
"""

import csv
import numpy as np
import scipy.io as sio
import cv2
from sklearn.tree import DecisionTreeClassifier  # Decision Tree 추가
from sklearn.preprocessing import StandardScaler

sampling_rate = 128
number_of_channel = 32  # considering only head electrodes
eeg_in_second = 63  # length of each trial
number_of_eeg = sampling_rate * eeg_in_second  # total inputs from a single channel

channel_names = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz',
                 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']


class PredictEmotion:
    """
    Receives EEG data preprocessing and predicts emotion using Decision Tree.
    """

    def __init__(self):
        """
        Initializes training data and trains Decision Tree classifiers for arousal and valence.
        """
        # Load training data
        self.train_features = self.get_csv("train_std.csv")
        self.class_arousal = self.get_csv("class_arousal.csv").ravel()  # 1D array로 변환
        self.class_valence = self.get_csv("class_valence.csv").ravel()

        # 데이터 스케일링 (정규화)
        self.scaler = StandardScaler()
        self.train_features = self.scaler.fit_transform(self.train_features)

        # Initialize Decision Tree classifiers
        self.tree_arousal = DecisionTreeClassifier(criterion="entropy", max_depth=10, random_state=42)
        self.tree_valence = DecisionTreeClassifier(criterion="entropy", max_depth=10, random_state=42)

        # Train the models
        self.tree_arousal.fit(self.train_features, self.class_arousal)
        self.tree_valence.fit(self.train_features, self.class_valence)

    def get_csv(self, path):
        """
        Load CSV file and convert to numpy array.
        """
        data_csv = np.loadtxt(path, delimiter=",", dtype=np.float64)
        return data_csv

    def get_feature(self, all_channel_data):
        """
        Extracts features (standard deviation) from EEG frequency bands.
        """
        delta, theta, alpha, beta, gamma = self.get_frequency(all_channel_data)

        delta_std = np.std(delta, axis=1)
        theta_std = np.std(theta, axis=1)
        alpha_std = np.std(alpha, axis=1)
        beta_std = np.std(beta, axis=1)
        gamma_std = np.std(gamma, axis=1)

        feature = np.concatenate([delta_std, theta_std, alpha_std, beta_std, gamma_std])
        return feature.reshape(1, -1)  # sklearn 입력 형태로 변환

    def get_frequency(self, all_channel_data):
        """
        Extracts EEG frequency bands: Delta, Theta, Alpha, Beta, Gamma.
        """
        L = all_channel_data.shape[1]
        Fs = 128
        data_fft = np.fft.fft(all_channel_data, axis=1)
        frequency = np.abs(data_fft / L)
        frequency = frequency[:, : L // 2 + 1]
        freq_bins = np.linspace(0, Fs / 2, L // 2 + 1)

        delta = frequency[:, (freq_bins >= 1) & (freq_bins < 4)]
        theta = frequency[:, (freq_bins >= 4) & (freq_bins < 8)]
        alpha = frequency[:, (freq_bins >= 8) & (freq_bins < 13)]
        beta = frequency[:, (freq_bins >= 13) & (freq_bins < 30)]
        gamma = frequency[:, (freq_bins >= 30) & (freq_bins < 50)]

        return delta, theta, alpha, beta, gamma

    def predict_emotion(self, feature):
        """
        Predicts arousal and valence using trained Decision Tree models.
        """
        feature = self.scaler.transform(feature)  # 스케일링 적용
        result_ar = self.tree_arousal.predict(feature)[0]
        result_va = self.tree_valence.predict(feature)[0]
        return result_ar, result_va

    def determine_emotion_class(self, feature):
        """
        Maps predicted arousal and valence to emotion classes based on Russell's Circumplex Model.
        """
        class_ar, class_va = self.predict_emotion(feature)

        if class_ar == 2.0 or class_va == 2.0:
            emotion_class = 5
        elif class_ar == 3.0 and class_va == 1.0:
            emotion_class = 1
        elif class_ar == 3.0 and class_va == 3.0:
            emotion_class = 2
        elif class_ar == 1.0 and class_va == 3.0:
            emotion_class = 3
        elif class_ar == 1.0 and class_va == 1.0:
            emotion_class = 4
        return emotion_class

    def process_all_data(self, all_channel_data):
        """
        Processes EEG data and predicts emotion class.
        """
        feature = self.get_feature(all_channel_data)
        emotion_class = self.determine_emotion_class(feature)
        print("Extracted feature vector:", feature)
        return emotion_class

    def main_process(self):
        """
        Loads EEG data, processes it, and predicts emotion.
        """
        fname = "../../DEAP/s01.mat"
        mat_data = sio.loadmat(fname)

        eeg_data = mat_data['data']
        labels = mat_data['labels']

        trial = 36
        eeg_realtime = eeg_data[trial, :, :]
        print("Labels for trial:", labels[trial, :])

        eeg_raw = eeg_realtime[:32, :]

        emotion_class = self.process_all_data(eeg_raw)
        print("Class of emotion =", emotion_class)

        emotion_dict = {
            1: "fear - nervous - stress - tense - upset",
            2: "happy - alert - excited - elated",
            3: "relax - calm - serene - contented",
            4: "sad - depressed - lethargic - fatigue",
            5: "Neutral"
        }
        print(f"Emotion: {emotion_dict.get(emotion_class, 'Unknown')}")

        return emotion_class


if __name__ == "__main__":
    rte = PredictEmotion()
    rte.main_process()

Labels for trial: [2.31 6.88 3.1  6.77]
Extracted feature vector: [[0.00912836 0.01054795 0.01253389 0.01554596 0.00755974 0.00765208
  0.01007174 0.01642988 0.00852434 0.00421333 0.01430973 0.00825571
  0.00699874 0.00881562 0.00826589 0.00566351 0.01360228 0.01296179
  0.00745096 0.01380788 0.01075683 0.00822561 0.00735315 0.00508146
  0.0083904  0.01266077 0.0042567  0.00480269 0.01798847 0.00433993
  0.01505433 0.0096854  0.05270875 0.06532088 0.07095523 0.06891861
  0.04619742 0.04795119 0.04411866 0.07050849 0.04528479 0.03380007
  0.06943404 0.05900324 0.05356046 0.05643514 0.0587685  0.03754868
  0.083211   0.07466218 0.11128249 0.09915445 0.05826971 0.04687831
  0.08355572 0.0361126  0.04319696 0.06949956 0.0358912  0.02879573
  0.07687482 0.0415526  0.07028759 0.06164409 0.04735012 0.04803978
  0.05242895 0.06221721 0.05024472 0.03580326 0.04406044 0.06492114
  0.05155443 0.03896031 0.06205272 0.06484133 0.07226919 0.05849023
  0.05499807 0.05000433 0.05286198 0.04877978 0.05

In [27]:
import pandas as pd
arousal_labels = pd.read_csv("class_arousal.csv", header=None).values.ravel()
valence_labels = pd.read_csv("class_valence.csv", header=None).values.ravel()
print("Arousal Label Distribution:", pd.Series(arousal_labels).value_counts())
print("Valence Label Distribution:", pd.Series(valence_labels).value_counts())

Arousal Label Distribution: 3    24
1    15
2     1
dtype: int64
Valence Label Distribution: 3    18
1    18
2     4
dtype: int64
